<a href="https://colab.research.google.com/github/agamhub/Spark-SQL/blob/main/SparkStandalone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup Spark

[Google Colab Documentation GCS](https://colab.research.google.com/notebooks/io.ipynb)





> **List Of To-Do**

✅ Spark DQC movement create multiple EXTERNAL table in 1 sessions of spark

✅ Connection to GCS

✅ Switch Postgres Airflow DB under docker to Local DB configuration

✅ PowerBI Airflow monitoring log







Setup Spark
Setup Apache Spark in 1️⃣ 2️⃣ 3️⃣ 4️⃣ steps (step 0️⃣ is the Java installation, which is skipped because Java is available in Google Colab).

The following code should also run on any Ubuntu machine or Docker container except for the Web servers links.

In [ ]:
import requests
import subprocess
import os
import re
import socket
import shutil
import time
import sys

def run(cmd):
    # run a shell command
    try:
        # Run the command and capture stdout and stderr
        subprocess_output = subprocess.run(cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        # Access stdout (stderr redirected to stdout)
        stdout_result = subprocess_output.stdout.strip().splitlines()[-1]
        # Process the results as needed
        print(f'✅ {stdout_result}')
        return stdout_result
    except subprocess.CalledProcessError as e:
        # Handle the error if the command returns a non-zero exit code
        print(f"Command failed with return code {e.returncode}")
        print("stdout:", e.stdout)

def is_java_installed():
    return shutil.which("java")

def install_java():
    # Uncomment and modify the desired version
    # java_version= 'openjdk-11-jre-headless'
    # java_version= 'default-jre'
    # java_version= 'openjdk-17-jre-headless'
    # java_version= 'openjdk-18-jre-headless'
    java_version= 'openjdk-19-jre-headless'
    os.environ['JAVA_HOME'] = ' /usr/lib/jvm/java-19-openjdk-amd64'
    print(f"Java not found. Installing {java_version} ... (this might take a while)")
    try:
        cmd = f"apt install -y {java_version}"
        subprocess_output = subprocess.run(cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
        stdout_result = subprocess_output.stdout
        # Process the results as needed
        print(f'✅ Done installing Java {java_version}')
    except subprocess.CalledProcessError as e:
        # Handle the error if the command returns a non-zero exit code
        print(f"Command failed with return code {e.returncode}")
        print("stdout:", e.stdout)

print("\n0️⃣   Install Java if not available")
if is_java_installed():
    print("✅ Java is already installed.")
else:
    install_java()

print("\n1️⃣   Download and install Hadoop and Spark")
# URL for downloading Hadoop and Spark
SPARK_VERSION = "3.5.4"
HADOOP_SPARK_URL = "https://dlcdn.apache.org/spark/spark-" + SPARK_VERSION + \
                   "/spark-" + SPARK_VERSION + "-bin-hadoop3.tgz"
r = requests.head(HADOOP_SPARK_URL)
if r.status_code >= 200 and r.status_code < 400:
    print(f'✅ {HADOOP_SPARK_URL} was found')
else:
    SPARK_CDN = "https://dlcdn.apache.org/spark/"
    print(f'⚠️ {HADOOP_SPARK_URL} was NOT found. \nCheck for available Spark versions in {SPARK_CDN}')

# set some environment variables
os.environ['SPARK_HOME'] = os.path.join(os.getcwd(), os.path.splitext(os.path.basename(HADOOP_SPARK_URL))[0])
os.environ['PATH'] = ':'.join([os.path.join(os.environ['SPARK_HOME'], 'bin'), os.environ['PATH']])
os.environ['PATH'] = ':'.join([os.path.join(os.environ['SPARK_HOME'], 'sbin'), os.environ['PATH']])

# download Spark
# using --no-clobber option will prevent wget from downloading file if already present
# shell command: wget --no-clobber $HADOOP_SPARK_URL
cmd = f"wget --no-clobber {HADOOP_SPARK_URL}"
run(cmd)

# uncompress
try:
    # Run the command and capture stdout and stderr
    cmd = "([ -d $(basename {0}|sed 's/\.[^.]*$//') ] && echo -n 'Folder already exists') || (tar xzf $(basename {0}) && echo 'Uncompressed Spark distribution')"
    subprocess_output = subprocess.run(cmd.format(HADOOP_SPARK_URL), shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    # Access stdout (stderr redirected to stdout)
    stdout_result = subprocess_output.stdout
    # Process the results as needed
    print(f'✅ {stdout_result}')

except subprocess.CalledProcessError as e:
    # Handle the error if the command returns a non-zero exit code
    print(f"Command failed with return code {e.returncode}")
    print("stdout:", e.stdout)


print("\n2️⃣   Start Spark engine")
# start master
# shell command: $SPARK_HOME/sbin/start-master.sh
cmd = os.path.join(os.environ['SPARK_HOME'], 'sbin', 'stop-master.sh')
run(cmd)
cmd = os.path.join(os.environ['SPARK_HOME'], 'sbin', 'start-master.sh')
out = run(cmd)

# start one worker (first stop it in case it's already running)
# shell command: $SPARK_HOME/sbin/start-worker.sh spark://${HOSTNAME}:7077
cmd = [os.path.join(os.environ['SPARK_HOME'], 'sbin', 'stop-worker.sh')]
run(cmd)
cmd = os.path.join(os.environ['SPARK_HOME'], 'sbin', 'start-worker.sh') + ' ' + 'spark://'+socket.gethostname()+':7077'
run(cmd)

print("\n3️⃣   Start Master Web UI")
# get master UI's port number
# the subprocess that's starting the master with start-master.sh
# might still not be ready with assigning the port number at this point
# therefore we check the logfile a few times (attempts=5) to see if the port
# has been assigned. This might take 1-2 seconds.

master_log = out.partition("logging to")[2].strip()
print("Search for port number in log file {}".format(master_log))
attempts = 10
search_pattern = "Successfully started service 'MasterUI' on port (\d+)"
found = False
for i in range(attempts):
  if not found:
   with open(master_log) as log:
      found = re.search(search_pattern, log.read())
      if found:
          webUIport = found.group(1)
          print(f"✅ Master UI is available at localhost:{webUIport} (attempt nr. {i})")
          break
      else:
          time.sleep(2) # need to try until port information is found in the logfile
          i+=1
if not found:
  print("Could not find port for Master Web UI\n")

IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    # serve the Web UI on Colab
    print("Click on the link below to open the Spark Web UI 🚀")
    from google.colab import output
    output.serve_kernel_port_as_window(webUIport)

print("\n4️⃣   Start history server")
# start history server
# shell command: mkdir -p /tmp/spark-events
# shell command: $SPARK_HOME/sbin/start-history-server.sh
spark_events_dir = os.path.join('/tmp', 'spark-events')
if not os.path.exists(spark_events_dir):
    os.mkdir(spark_events_dir)
cmd = os.path.join(os.environ['SPARK_HOME'], 'sbin', 'stop-history-server.sh')
run(cmd)
cmd = os.path.join(os.environ['SPARK_HOME'], 'sbin', 'start-history-server.sh')
run(cmd)

if IN_COLAB:
    # serve the History Server
    print("Click on the link below to open the Spark History Server Web UI 🚀")
    output.serve_kernel_port_as_window(18080)



0️⃣   Install Java if not available
✅ Java is already installed.

1️⃣   Download and install Hadoop and Spark
✅ https://dlcdn.apache.org/spark/spark-3.5.4/spark-3.5.4-bin-hadoop3.tgz was found
✅ File ‘spark-3.5.4-bin-hadoop3.tgz’ already there; not retrieving.
✅ Folder already exists

2️⃣   Start Spark engine
✅ stopping org.apache.spark.deploy.master.Master
✅ starting org.apache.spark.deploy.master.Master, logging to /content/spark-3.5.4-bin-hadoop3/logs/spark--org.apache.spark.deploy.master.Master-1-e692a16b1cfc.out
✅ stopping org.apache.spark.deploy.worker.Worker
✅ starting org.apache.spark.deploy.worker.Worker, logging to /content/spark-3.5.4-bin-hadoop3/logs/spark--org.apache.spark.deploy.worker.Worker-1-e692a16b1cfc.out

3️⃣   Start Master Web UI
Search for port number in log file /content/spark-3.5.4-bin-hadoop3/logs/spark--org.apache.spark.deploy.master.Master-1-e692a16b1cfc.out
✅ Master UI is available at localhost:8081 (attempt nr. 2)
Click on the link below to open the Spark

<IPython.core.display.Javascript object>


4️⃣   Start history server
✅ stopping org.apache.spark.deploy.history.HistoryServer
✅ starting org.apache.spark.deploy.history.HistoryServer, logging to /content/spark-3.5.4-bin-hadoop3/logs/spark--org.apache.spark.deploy.history.HistoryServer-1-e692a16b1cfc.out
Click on the link below to open the Spark History Server Web UI 🚀
Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>

# Start and Stop Worker



In [ ]:
cmd = [os.path.join(os.environ['SPARK_HOME'], 'sbin', 'stop-worker.sh')]
run(cmd)
cmd = os.path.join(os.environ['SPARK_HOME'], 'sbin', 'start-worker.sh') + ' ' + 'spark://'+socket.gethostname()+':7077'
run(cmd)

✅ stopping org.apache.spark.deploy.worker.Worker
✅ starting org.apache.spark.deploy.worker.Worker, logging to /content/spark-3.5.4-bin-hadoop3/logs/spark--org.apache.spark.deploy.worker.Worker-1-e692a16b1cfc.out


'starting org.apache.spark.deploy.worker.Worker, logging to /content/spark-3.5.4-bin-hadoop3/logs/spark--org.apache.spark.deploy.worker.Worker-1-e692a16b1cfc.out'

# Spark with python & java examples

In [ ]:
%%bash

EXAMPLES_JAR=$(find $SPARK_HOME/examples/jars/ -name "spark-examples*")

$SPARK_HOME/bin/spark-submit \
  --class org.apache.spark.examples.SparkPi \
  --master spark://${HOSTNAME}:7077 \
  --conf spark.eventLog.enabled=true \
  $EXAMPLES_JAR \
  100 \
  2>/tmp/SparkPi_bash.log

Pi is roughly 3.1413179141317915


In [ ]:
%%bash

EXAMPLES_PY=$(find $SPARK_HOME/examples/src/main/python/ -name "wordcount.py")

echo $EXAMPLES_PY

file=("/content/wordcount.txt")

echo $file

$SPARK_HOME/bin/spark-submit \
  --master spark://${HOSTNAME}:7077 \
  --name "GOOGLECOLAB" \
  --driver-memory 1g \
  --num-executors 1 \
  --executor-memory 1024M \
  --conf spark.cores.max=2 \
  $EXAMPLES_PY $file 2> /tmp/pyPI.log

/content/spark-3.5.4-bin-hadoop3/examples/src/main/python/wordcount.py
/content/wordcount.txt
halo: 1
nama: 1
saya: 1
agam: 1
adhinegara: 1


In [ ]:
NGROK = True #@param {type:"boolean"}

In [ ]:
if NGROK:
  !pip install pyngrok
  from pyngrok import ngrok, conf
  import getpass

  print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken")
  authtoken = getpass.getpass()
  conf.get_default().auth_token = authtoken

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken
··········


In [ ]:
if NGROK:
  # close all existing connections (https://pyngrok.readthedocs.io/en/latest/#get-active-tunnels)
  tunnels = ngrok.get_tunnels()
  if tunnels:
    map(lambda t: ngrok.disconnect(t.public_url), tunnels)
  # Open a ngrok tunnel to the HTTP server
  public_url = ngrok.connect(8081).public_url
  print(f'Click on {public_url} to open the Spark Master Web UI')

Click on https://a839-34-135-15-192.ngrok-free.app to open the Spark Master Web UI


In [ ]:
if NGROK:
  # Open a ngrok tunnel to the HTTP server
  public_url = ngrok.connect(18080).public_url
  print(f'Click on {public_url} to open the Spark Master Web UI')

Click on https://765f-34-135-15-192.ngrok-free.app to open the Spark Master Web UI
